In [ ]:
#preliminary models to see which ones might provide the best scores

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
all_drugs = pd.read_csv('./data/all_drugs.csv')

C:\Users\12155\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
all_drugs.shape

In [ ]:
all_drugs.columns

In [3]:
all_drugs.drop(columns=['Corresponding_Generic_Drug_Effective_Date', 'Unnamed: 0', 'DF;Route'], inplace=True)

In [4]:
features =  all_drugs[['NDC Description','Pharmacy_Type_Indicator', 'Classification_for_Rate_Setting', 'dosage_form', 'route',
           'Ingredient', 'Type']].copy()
feature_dummies = pd.get_dummies(features, columns = ['NDC Description','Pharmacy_Type_Indicator', 'Classification_for_Rate_Setting', 'dosage_form', 'route',
           'Ingredient', 'Type'], drop_first=True)
X = feature_dummies
y = all_drugs['NADAC_Per_Unit']

In [5]:
feature_dummies

,NDC Description_ACETAMINOPHEN 325MG TABLET,NDC Description_ACETAMINOPHEN ER 650MG TABLET,NDC Description_ACETAZOLAMIDE 125MG TABLET,NDC Description_ACETAZOLAMIDE 250MG TABLET,NDC Description_ACTIVELLA 1MG-0.5MG TABLET,NDC Description_ACYCLOVIR 400MG TABLET,NDC Description_ACYCLOVIR 800MG TABLET,NDC Description_ADEFOVIR DIPIVOXIL 10MG TABLET ORAL,NDC Description_AFEDITAB CR 30MG TABLET,NDC Description_AFEDITAB CR 60MG TABLET,...,Ingredient_TROSPIUM CHLORIDE,Ingredient_VALPROIC ACID,Ingredient_VALSARTAN,Ingredient_VORICONAZOLE,Ingredient_ZAFIRLUKAST,Ingredient_ZIDOVUDINE,Ingredient_ZOLMITRIPTAN,Ingredient_ZOLPIDEM TARTRATE,Type_OTC,Type_RX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126972,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
126973,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
126974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
126975,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
#linear regression
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.3, random_state = 17)
lr = LinearRegression()
lr.fit(X_train, y_train)
print(f'LR Train: {lr.score(X_train, y_train)}')
print(f'LR Test: {lr.score(X_test, y_test)}')

LR Train: 0.926903481896789
LR Test: -6.147243174148271e+18


In [7]:
test_preds = lr.predict(X_test)
train_preds = lr.predict(X_train)
print(f'Test RMSE: {metrics.mean_squared_error(y_test, test_preds, squared=False)}')


Test RMSE: 33619121272.395153


In [ ]:
print(f'Train RMSE: {metrics.mean_squared_error(y_train, train_preds, squared=False)}')


In [ ]:
# LR Train: 0.926903481896789
# LR Test: -6.147243174148271e+18
# Train RMSE: 3.1971220088093455
# Test RMSE: 33619121272.395153

In [ ]:
#knn
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor

sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)


In [ ]:
knn = KNeighborsRegressor(n_neighbors = 7)

In [ ]:
knn.fit(X_train_sc, y_train)
print(f'Train Score {knn.score(X_train_sc, y_train)}')
print(f'Test Score {knn.score(X_test_sc, y_test)}')

In [ ]:
y_pred = knn.predict(X_test_sc)

In [ ]:
print(f'Test RMSE {metrics.mean_squared_error(y_test, y_pred, squared= False)}')

In [ ]:
#Train Score 0.8915498524974116
#Test Score 0.8080358209686297
#Test RMSE 5.940954948304041

In [9]:
#random forest
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.model_selection import cross_val_score

rf = RandomForestRegressor()
et = ExtraTreesRegressor()

In [ ]:
#ran this once then accidentally restarted. I interrupted the kernal, but the best params are in a cell below in a rf model
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [95,100,105],
    'max_features': ['auto'],
    'max_depth': [3,4,5,6]
}

gs = GridSearchCV(rf, param_grid = params, cv=5)
gs.fit(X_train_sc, y_train)
gs.best_score_

In [ ]:
gs.best_params_

In [10]:
rt = RandomForestRegressor(max_depth = 6, max_features='auto', n_estimators = 105)
rt.fit(X_train, y_train)
print(rt.score(X_train, y_train))
print(rt.score(X_test, y_test))
#train is better but test is slightly worse

0.8735370923127105
0.7982807359902575


In [11]:
y_preds_rt = rt.predict(X_test)
print(metrics.mean_squared_error(y_test, y_preds_rt, squared=False))
#slightly worse than KNN

6.0900358410633615


In [ ]:
#adaboost
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor

vote = VotingRegressor([
    ('ada', AdaBoostRegressor()),
    ('gb', GradientBoostingRegressor()),
    ('forest', RandomForestRegressor())
])

params = {
    'ada__n_estimators':[100,150,200],
    'gb__n_estimators':[100,150,200],
    'forest__max_depth':[6,7,8]
}

gs = GridSearchCV(vote, param_grid = params, cv=5)
gs.fit(X_train_sc, y_train)
print(gs.best_score_)
print(gs.best_params_)
#best score was 0.29 something (lost connection before I could put it in a comment)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor(n_estimators = 100)

ada.fit(X_train_sc, y_train)


In [ ]:
print(f'ADA Train Score {ada.score(X_train_sc, y_train)}')
print(f'ADA Test Score {ada.score(X_test_sc, y_test)}')
# ADA Train Score 0.6778278098461128
# ADA Test Score 0.6458744617545746

In [ ]:
ada_preds = ada.predictict(X_test_sc)
print(f'RMSE: {metrics.mean_squared_error(y_test, ada_preds, squared=False)}')
# RMSE: 8.069094188341497

In [ ]:
#svm 
from sklearn.svm import LinearSVR

# pgrid = {"C": np.linspace(0.0001, 1, 20)}

# gs = GridSearchCV(LinearSVR(), param_grid = pgrid, cv=5)

# gs.fit(X_train_sc, y_train)


In [ ]:
gs.best_params_
#{'C': 0.4737368421052632}

In [ ]:
svr = LinearSVR(C = 0.4737368421052632 )

svr.fit(X_train_sc, y_train)

In [ ]:
print(f'SVR Train {svr.score(X_train_sc, y_train)}')
print(f'SVR Test {svr.score(X_test_sc, y_test)}')
# SVR Train 0.9113543781178794
# SVR Test 0.8165252558438779

In [ ]:
svr_preds = svr.predict(X_test_sc)
print(f'RMSE: {metrics.mean_squared_error(y_test, svr_preds, squared=False)}')
# RMSE: 5.8081029598961464